# Helper functions for the Autoencoder model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from IPython.display import display

In [ ]:
# Path to save model weight
ae_paths = ["../models/AE_1c", "../models/AE_2c", "../models/AE_3c"]

# Batch Size
batch_size = 256

# Shuffling Parameter for Pipeline
shuffle = 1024

# Encoder Input Dimension
input_shape_img = (64, 64, 7)
input_shape_stats = (4,)

# Dropout rate for dense layers
dropout = 0.5

# Encoder Output dimension - Decoder Input Dimension
latent_z_dim_labels = 5

# Decoder latent dimension
latent_dim = (1, 1, 2048)

# Number of Training Epochs
epochs = 50

In [ ]:
def load_data(set=1, test=False):
    # Path for the Traning and Validation Data
    dataPath = [
        "../data/data_v1.npz",
        "../data/data_v2.npz",
        "../data/data_v3.npz",
        "../data/snr30.npz",
        "../data/snr60.npz",
    ]

    # Load dataset
    with np.load(dataPath[set - 1]) as data:
        image = data["img"]
        image_nonoise = data["img_nonoise"]
        label = data["label"]
        snr = data["snr"]
        sigma = data["sigma"]
        psf = data["psf_r"]
        psf_img = data["psf_img"]

    # Compute Image Statistics (StdDev & Mean of the pixel values of every image)
    stats = image.std(axis=(1, 2))
    mean = image.mean(axis=(1, 2))

    if not test:
        # 90% of the data to be kept for training
        n_train = int(snr.shape[0] * 0.9)

        # Divide PSF Images into training and validation depending on dataset
        # Dataset 1 & 2 have fixed PSF and hence only one PSF image
        if set == 3:
            psf_img_tr, psf_img_val = psf_img[:n_train], psf_img[n_train:]
        else:
            psf_img_tr, psf_img_val = psf_img, psf_img

        # Return Training and Validation Datasets
        return dict(
            image_train=image[:n_train],
            image_val=image[n_train:],
            image_nonoise_train=image_nonoise[:n_train],
            image_nonoise_val=image_nonoise[n_train:],
            label_train=label[:n_train],
            label_val=label[n_train:],
            psf_train=psf[:n_train],
            psf_val=psf[n_train:],
            snr_train=snr[:n_train],
            snr_val=snr[n_train:],
            sigma_train=sigma[:n_train],
            sigma_val=sigma[n_train:],
            psf_img_train=psf_img_tr,
            psf_img_val=psf_img_val,
            stats_train=stats[:n_train],
            stats_val=stats[n_train:],
            mean_train=mean[:n_train],
            mean_val=mean[n_train:],
        )

    else:
        return dict(
            image_test=image,
            image_nonoise_test=image_nonoise,
            label_test=label,
            psf_test=psf,
            snr_test=snr,
            sigma_test=sigma,
            psf_img_test=psf_img,
            stats_test=stats,
            mean_test=mean,
        )

In [ ]:
def create_dataset(data, set=1, test=False):
    if not test:
        # Number of elements in Training and Validation set
        n_train = data["sigma_train"].shape[0]
        n_val = data["sigma_val"].shape[0]

        # PSF Image handling
        if set != 3:
            psf_img_tr = np.array(list(data["psf_img_train"]) * n_train)
            psf_img_v = np.array(list(data["psf_img_train"]) * n_val)
        else:
            psf_img_tr = data["psf_img_train"]
            psf_img_v = data["psf_img_val"]

        # Reshape the Parameter
        sigma_t, sigma_v = (data["sigma_train"].reshape(-1, 1), data["sigma_val"].reshape(-1, 1))
        psf_t, psf_v = (data["psf_train"].reshape(-1, 1), data["psf_val"].reshape(-1, 1))
        stats_t, stats_v = (data["stats_train"].reshape(-1, 1), data["stats_val"].reshape(-1, 1))
        mean_t, mean_v = (data["mean_train"].reshape(-1, 1), data["mean_val"].reshape(-1, 1))

        # Create Training Dataset
        training = tf.data.Dataset.from_tensor_slices(
            {
                "Image": data["image_train"],
                "No-noise Image": data["image_nonoise_train"],
                "PSF_img": psf_img_tr,
                "Labels": data["label_train"],
                "Variance": data["stats_train"][:, np.newaxis, np.newaxis] ** 2,
                "Sigma": data["sigma_train"][:, np.newaxis, np.newaxis],
                "Stats": np.hstack([sigma_t, psf_t, stats_t, mean_t]),
            }
        )

        # Create Validation Dataset
        validation = tf.data.Dataset.from_tensor_slices(
            {
                "Image": data["image_val"],
                "No-noise Image": data["image_nonoise_val"],
                "PSF_img": psf_img_v,
                "Labels": data["label_val"],
                "Variance": data["stats_val"][:, np.newaxis, np.newaxis] ** 2,
                "Sigma": data["sigma_val"][:, np.newaxis, np.newaxis],
                "Stats": np.hstack([sigma_v, psf_v, stats_v, mean_v]),
            }
        )

        return training, validation

    else:
        n_test = data["sigma_test"].shape[0]
        psf_img_te = np.array(list(data["psf_img_test"]) * n_test)
        sigma_t = data["sigma_test"].reshape(-1, 1)
        psf_t = data["psf_test"].reshape(-1, 1)
        stats_t = data["stats_test"].reshape(-1, 1)
        mean_t = data["mean_test"].reshape(-1, 1)

        # Create testing Dataset
        testing = tf.data.Dataset.from_tensor_slices(
            {
                "Image": data["image_test"],
                "No-noise Image": data["image_nonoise_test"],
                "PSF_img": psf_img_te,
                "Labels": data["label_test"],
                "Variance": data["stats_test"][:, np.newaxis, np.newaxis] ** 2,
                "Sigma": data["sigma_test"][:, np.newaxis, np.newaxis],
                "Stats": np.hstack([sigma_t, psf_t, stats_t, mean_t]),
            }
        )

        return testing

In [ ]:
def pipeline_noisy_image_to_clean_image(element):
    image = element["Image"]
    psf = element["PSF_img"]

    sigma = element["Sigma"]
    var = element["Variance"]
    stats = element["Stats"]

    # Preprocessing of Images
    img_sig = image / sigma
    img_sig_sq = image ** 2 / (1.41 * sigma ** 2)
    img_var = image / var
    img_var_sq = image / (var ** 2 * sigma)
    psf_sq = psf ** 2

    clean = element["No-noise Image"]
    label = tf.cast(element["Labels"], tf.float32)

    # Seven channel image
    img = tf.stack([image, img_sig, img_sig_sq, img_var, img_var_sq, psf, psf_sq], axis=-1)

    return ((img, stats), (clean, label))

In [ ]:
# Function to scale labels
def norm_label(label_train, label_val=None, scaler=None):
    if not scaler:
        scaler = MinMaxScaler(feature_range=(0, 1))
        label_train = scaler.fit_transform(label_train)
        label_val = scaler.transform(label_val)
        return label_train, label_val, scaler

    else:
        return scaler.transform(label_train)


# Function to unscale labels
def unnorm_label(label, scaler):
    label = scaler.inverse_transform(label)

    return label

In [ ]:
# Function to scale images
def norm_image(image_train, image_val=None, test=False, scaler=None):
    if not test:
        min_pixel = image_train.min()
        max_pixel = image_train.max()
        diff = max_pixel - min_pixel

        image_train = ((image_train - min_pixel) / diff)[:, :, :, np.newaxis]
        image_val = ((image_val - min_pixel) / diff)[:, :, :, np.newaxis]

        return image_train, image_val, (min_pixel, diff)

    else:
        min_pixel = scaler[0]
        diff = scaler[1]

        image_train = ((image_train - min_pixel) / diff)[:, :, :, np.newaxis]

        return image_train

In [ ]:
class ConvEncoder(layers.Layer):
    """
    Convolutional Encoder Layer Class.
    Converts an input into a latent representation.
    """

    def __init__(self, input_shape, dropout_rate=0.0, name="encoder", **kwargs):
        """
        Initializes the encoder layers and saves them as local attribute.
        
        Input:
        -input_dim: 3D-tuple with (rows, cols, channels) input image dimensions.
        
        Returns nothing.
        """
        super(ConvEncoder, self).__init__(name=name, input_shape=input_shape, **kwargs)

        self.conv1 = layers.Conv2D(filters=32, kernel_size=(3, 3), padding="same", strides=(2, 2))
        self.lRelu1 = layers.LeakyReLU()
        self.drop1 = layers.Dropout(dropout_rate)

        self.conv2 = layers.Conv2D(filters=64, kernel_size=(3, 3), padding="same", strides=(2, 2))
        self.lRelu2 = layers.LeakyReLU()
        self.drop2 = layers.Dropout(dropout_rate)

        self.conv3 = layers.Conv2D(filters=128, kernel_size=(3, 3), padding="same", strides=(2, 2))
        self.lRelu3 = layers.LeakyReLU()
        self.drop3 = layers.Dropout(dropout_rate)

        self.conv4 = layers.Conv2D(filters=256, kernel_size=(3, 3), padding="same", strides=(2, 2))
        self.lRelu4 = layers.LeakyReLU()
        self.drop4 = layers.Dropout(dropout_rate)

        self.conv5 = layers.Conv2D(filters=512, kernel_size=(3, 3), padding="same", strides=(2, 2))
        self.lRelu5 = layers.LeakyReLU()
        self.drop5 = layers.Dropout(dropout_rate)

        self.conv6 = layers.Conv2D(filters=2048, kernel_size=(3, 3), padding="same", strides=(2, 2))
        self.lRelu6 = layers.LeakyReLU()
        self.drop6 = layers.Dropout(dropout_rate)

    def call(self, inputs, training=None):
        """
        Runs the encoding inference for `inputs`.
        
        Inputs:
        -inputs: 4D-tensor with dimension (batch_size, self.input_dim).
        """

        z = self.conv1(inputs)
        z = self.lRelu1(z)
        z = self.drop1(z)

        z = self.conv2(z)
        z = self.lRelu2(z)
        z = self.drop2(z)

        z = self.conv3(z)
        z = self.lRelu3(z)
        z = self.drop3(z)

        z = self.conv4(z)
        z = self.lRelu4(z)
        z = self.drop4(z)

        z = self.conv5(z)
        z = self.lRelu5(z)
        z = self.drop5(z)

        z = self.conv6(z)
        z = self.lRelu6(z)
        z = self.drop6(z)

        return z

In [ ]:
class ConvDecoder(layers.Layer):
    """
    Convolutional Decoder Layer Class.
    Converts z, the encoded digit vector, back into a readable digit.
    """

    def __init__(self, input_shape, dropout_rate=0.0, name="decoder", **kwargs):
        """
        Initializes the decoder architecture and saves it as a local attribute.
        
        Input:
        -input_shape: 3D-tuple with (rows, cols, channels) input representation.
        
        Returns nothing.
        """
        super(ConvDecoder, self).__init__(name=name, input_shape=input_shape, **kwargs)

        self.convT1 = layers.Conv2DTranspose(512, kernel_size=(3, 3), strides=(2, 2), padding="same")
        self.lRelu1 = layers.LeakyReLU()
        self.drop1 = layers.Dropout(dropout_rate)

        self.convT2 = layers.Conv2DTranspose(256, kernel_size=(3, 3), strides=(2, 2), padding="same")
        self.lRelu2 = layers.LeakyReLU()
        self.drop2 = layers.Dropout(dropout_rate)

        self.convT3 = layers.Conv2DTranspose(128, kernel_size=(3, 3), strides=(2, 2), padding="same")
        self.lRelu3 = layers.LeakyReLU()
        self.drop3 = layers.Dropout(dropout_rate)

        self.convT4 = layers.Conv2DTranspose(64, kernel_size=(3, 3), strides=(2, 2), padding="same")
        self.lRelu4 = layers.LeakyReLU()
        self.drop4 = layers.Dropout(dropout_rate)

        self.convT5 = layers.Conv2DTranspose(32, kernel_size=(3, 3), strides=(2, 2), padding="same")
        self.lRelu5 = layers.LeakyReLU()
        self.drop5 = layers.Dropout(dropout_rate)

        self.convT6 = layers.Conv2DTranspose(
            1, kernel_size=(3, 3), strides=(2, 2), padding="same", activation="sigmoid"
        )

    def call(self, inputs, training=None):
        """
        Runs the encoding inference for `inputs`.
        
        Inputs:
        -inputs: 4D-tensor with dimension (batch_size, self.input_dim).
        """

        x = self.convT1(inputs)
        x = self.lRelu1(x)
        x = self.drop1(x)

        x = self.convT2(x)
        x = self.lRelu2(x)
        x = self.drop2(x)

        x = self.convT3(x)
        x = self.lRelu3(x)
        x = self.drop3(x)

        x = self.convT4(x)
        x = self.lRelu4(x)
        x = self.drop4(x)

        x = self.convT5(x)
        x = self.lRelu5(x)
        x = self.drop5(x)

        x = self.convT6(x)

        return x

In [ ]:
# Create Model
def create_encoder(loss="binary_crossentropy", summary=True):

    # Create Encoder
    input_encoder_img = layers.Input(input_shape_img)
    input_encoder_stats = layers.Input(input_shape_stats)

    norm_encoder_img = layers.BatchNormalization()(input_encoder_img)
    norm_encoder_stats = layers.BatchNormalization()(input_encoder_stats)

    x = ConvEncoder(input_shape_img)(norm_encoder_img)
    x = layers.Flatten()(x)

    x = layers.concatenate([x, norm_encoder_stats])

    x = layers.Dense(2048)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(dropout)(x)

    x = layers.Dense(1024)(x)
    x = layers.Dropout(dropout)(x)

    # Output later of the encoder
    latent_z = layers.Dense(latent_z_dim_labels, activation="sigmoid")(x)

    # Encoder Model
    encoder = tf.keras.Model([input_encoder_img, input_encoder_stats], latent_z, name="Encoder")

    # Define Optimizer
    optimizer = optimizers.Adam(learning_rate=0.001)

    # Compile the model
    encoder.compile(optimizer, loss=loss)

    # Display the model summary
    if summary:
        display(encoder.summary())

    # Return encoder
    return encoder

In [ ]:
# Create Decoder Model
def create_decoder(loss="binary_crossentropy", summary=True):

    # Create Decoder
    input_decoder_labels = layers.Input(latent_z_dim_labels)

    x = layers.Dense(1024)(input_decoder_labels)
    x = layers.Dropout(dropout)(x)

    x = layers.Dense(2048)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(dropout)(x)

    x = layers.Reshape(latent_dim)(x)
    recon = ConvDecoder(latent_dim)(x)

    # Decoder Model
    decoder = tf.keras.Model(input_decoder_labels, recon, name="Decoder")

    # Define Optimizer
    optimizer = optimizers.Adam(learning_rate=0.001)

    # Compile the model
    decoder.compile(optimizer, loss=loss)

    # Display the model summary
    if summary:
        display(decoder.summary())

    # Return decoder
    return decoder

In [ ]:
def create_AE(encoder, decoder, summary=True):
    input_encoder_img = layers.Input(input_shape_img)
    input_encoder_stats = layers.Input(input_shape_stats)

    labels = encoder([input_encoder_img, input_encoder_stats])
    recons = decoder(labels)

    # AE Model
    AE = tf.keras.Model([input_encoder_img, input_encoder_stats], [recons, labels], name="AE")

    # Define Optimizer
    optimizer = optimizers.Adam(learning_rate=0.0001)

    # Define Loss
    loss = {"Encoder": "binary_crossentropy", "Decoder": "binary_crossentropy"}

    # Define Loss
    lossWeights = {"Encoder": 5.0, "Decoder": 1.0}

    # Compile the model
    AE.compile(optimizer, loss=loss, loss_weights=lossWeights)

    # Display the model summary
    if summary:
        display(AE.summary())

    # Return reverse decoder
    return AE

In [ ]:
def train_or_load_AE(AE, set=1, train=False, tr_ds_AE=None, val_ds_AE=None):
    # Scheduler for learning rate
    def scheduler(epoch):
        if epoch < 5:
            return 0.001
        else:
            return 0.001 * np.exp(0.1 * (5 - epoch))

    if not train:
        AE.load_weights(ae_paths[set - 1])
        AE.trainable = False

    else:
        AE.trainable = True
        history = AE.fit(
            tr_ds_AE,
            epochs=epochs,
            verbose=1,
            validation_data=val_ds_AE,
            callbacks=[
                EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True),
                LearningRateScheduler(scheduler),
            ],
        )

        AE.save_weights(ae_paths[set - 1])
        AE.trainable = False

    return AE